In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [27]:
from nlpsig_networks.scripts.ffn_baseline_functions import obtain_signatures_history

x_data = obtain_signatures_history(method="umap",
                              dimension=10,
                              sig_depth=3,
                              log_signature=True,
                              df=df_rumours,
                              id_column="timeline_id",
                              label_column="label",
                              embeddings=sbert_embeddings.numpy(),
                              seed=seed,
                              path_indices=None,
                              concatenate_current=False)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [01:10<00:00, 78.87it/s] 


In [32]:
x_data.shape

torch.Size([5568, 1110])

# Baseline: FFN using signatures

In [7]:
dim_reduce_methods = ["umap", "gaussian_random_projection"]
num_epochs = 100
batch=64
log_sig = True
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3)]#[(28, 2), (10, 3), (6, 4)]
hidden_dim_sizes = [[64,64],[128,128], [256,256]]#[[64,64], [128,128], [256,256]]
dropout_rates =  [0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
patience=5
validation_metric = "f1"

In [8]:
ffn_logsignature_umap, best_ffn_logsignature_umap, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    log_signature=log_sig,
    dim_reduce_methods=dim_reduce_methods,
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths, 
    path_indices=None,
    #data_split_seed=0,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/2 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.





























































































































































































































































































































































































































































































































































































































































100%|██████████| 5568/5568 [01:12<00:00, 76.53it/s]















100%|██████████| 3/3 [07:59<00:00, 159.68s/it]














100%|██████████| 3/3 [06:29<00:00, 129.96s/it]


100%|██████████| 2/2 [14:28<00:00, 434.46s/it]















100%|██████████| 3/3 [04:51<00:00, 97.14s/it]














100%|██████████| 3/3 [03:05<00:00, 61.69s/it]


100%|██████████| 2

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.






































































































































































































































































































































































































































































































































































































































































100%|██████████| 5568/5568 [01:14<00:00, 75.17it/s]















100%|██████████| 3/3 [02:43<00:00, 54.46s/it]














100%|██████████| 3/3 [02:58<00:00, 59.36s/it]


100%|██████████| 2/2 [05:41<00:00, 170.74s/it]















100%|██████████| 3/3 [03:23<00:00, 67.93s/it]














100%|██████████| 3/3 [03:05<00:00, 61.89s/it]


100%|██████

...











































































































































































































































































































































































































































































































































































































































100%|██████████| 5568/5568 [01:12<00:00, 76.42it/s]















100%|██████████| 3/3 [09:58<00:00, 199.47s/it]














100%|██████████| 3/3 [03:02<00:00, 60.98s/it]


100%|██████████| 2/2 [13:01<00:00, 390.68s/it]















100%|██████████| 3/3 [04:17<00:00, 85.92s/it]














100%|██████████| 3/3 [04:12<00:00, 84.15s/it]


100%|██████████| 2/2 [08:30<00:00, 2

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.









































































































































































































































































































































































































































































































































































































































100%|██████████| 5568/5568 [01:12<00:00, 77.10it/s]















100%|██████████| 3/3 [03:16<00:00, 65.41s/it]














100%|██████████| 3/3 [03:29<00:00, 69.69s/it]


100%|██████████| 2/2 [06:45<00:00, 202.66s/it]















100%|██████████| 3/3 [03:31<00:00, 70.61s/it]














100%|██████████| 3/3 [02:56<00:00, 58.93s/it]


100%|██████████| 2/2 [06:28<00:00, 194.3

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [01:15<00:00, 74.10it/s] 


In [9]:
ffn_logsignature_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,gamma,k_fold,n_splits,batch_size,model_id,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.560534,0.530334,"[0.6494296577946768, 0.4112388250319285]",0.563173,"[0.5574412532637075, 0.568904593639576]",0.549889,"[0.7777777777777778, 0.322]",0.241048,0.644128,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.523356,0.343554,"[0.6871088861076345, 0.0]",0.261678,"[0.5233555767397522, 0.0]",0.500000,"[1.0, 0.0]",0.240474,0.637011,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.506196,0.450859,"[0.6251808972503619, 0.276536312849162]",0.488470,"[0.5186074429771909, 0.4583333333333333]",0.492443,"[0.7868852459016393, 0.198]",0.237801,0.633452,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.515729,0.500869,"[0.4147465437788018, 0.5869918699186992]",0.529392,"[0.5642633228840125, 0.4945205479452055]",0.524934,"[0.32786885245901637, 0.722]",0.234720,0.690391,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.523356,0.343554,"[0.6871088861076345, 0.0]",0.261678,"[0.5233555767397522, 0.0]",0.500000,"[1.0, 0.0]",0.239666,0.637011,...,2,False,None,64,0.00,1196,28,2,umap,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.594852,0.594674,"[0.5861733203505355, 0.6031746031746031]",0.597691,"[0.6297071129707112, 0.5656742556917689]",0.597135,"[0.5482695810564663, 0.646]",0.233763,0.665480,...,2,False,None,64,3.17,1494,10,3,gaussian_random_projection,True
0,focal,0.586273,0.586263,"[0.5842911877394636, 0.588235294117647]",0.587864,"[0.6161616161616161, 0.5595667870036101]",0.587778,"[0.5555555555555556, 0.62]",0.246209,0.693950,...,2,False,None,64,3.17,1494,10,3,gaussian_random_projection,True
0,focal,0.540515,0.539154,"[0.5141129032258065, 0.5641952983725135]",0.545236,"[0.5756207674943566, 0.5148514851485149]",0.544240,"[0.4644808743169399, 0.624]",0.262308,0.665480,...,2,False,None,64,3.17,1494,10,3,gaussian_random_projection,True
0,focal,0.582460,0.581470,"[0.6018181818181818, 0.5611222444889781]",0.581487,"[0.6007259528130672, 0.5622489959839357]",0.581457,"[0.6029143897996357, 0.56]",0.538170,0.647687,...,2,False,None,64,3.17,1494,10,3,gaussian_random_projection,True


In [10]:
best_ffn_logsignature_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.537655,0.534071,"[0.49320794148380354, 0.574934268185802]",0.545066,"[0.5784313725490197, 0.5117004680187207]",0.542936,"[0.42987249544626593, 0.656]",0.234350,0.725979,...,0,2,False,None,64,1196,28,2,umap,True
0,focal,0.505243,0.496271,"[0.429042904290429, 0.5634987384356602]",0.513939,"[0.5416666666666666, 0.4862119013062409]",0.512596,"[0.3551912568306011, 0.67]",0.229483,0.633452,...,1,2,False,None,64,1196,28,2,umap,True
0,focal,0.546234,0.538361,"[0.4780701754385965, 0.5986509274873524]",0.559022,"[0.6005509641873278, 0.5174927113702624]",0.553543,"[0.3970856102003643, 0.71]",0.231095,0.690391,...,12,2,False,None,64,1196,28,2,umap,True
0,focal,0.488084,0.477972,"[0.4053156146179402, 0.5506276150627616]",0.495165,"[0.5169491525423728, 0.47338129496402875]",0.495667,"[0.3333333333333333, 0.658]",0.233398,0.672598,...,123,2,False,None,64,1196,28,2,umap,True
0,focal,0.538608,0.528846,"[0.4610244988864142, 0.5966666666666667]",0.552276,"[0.5931232091690545, 0.5114285714285715]",0.546525,"[0.3770491803278688, 0.716]",0.230608,0.690391,...,1234,2,False,None,64,1196,28,2,umap,True


In [11]:
best_ffn_logsignature_umap["valid_f1"].mean()

0.653634782090473

In [12]:
best_ffn_logsignature_umap["f1"].mean()

0.5151039350555426

In [16]:
best_ffn_logsignature_umap["precision"].mean()

0.5330936312202266

In [17]:
best_ffn_logsignature_umap["recall"].mean()

0.5302531876138434

In [18]:
np.stack(best_ffn_logsignature_umap["f1_scores"]).mean(axis=0)

array([0.45333223, 0.57687564])